In [4]:
from scipy.spatial.transform import Rotation as R
import numpy as np

In [8]:
r = [-0.15178555, 0.37758884, 0.27728027, 0.8703467]
t = [-0.14371057, -3.6296544, 2.1912692]

In [9]:
r = R.from_quat(r)
r = r.as_matrix()
r

array([[ 0.5610846 , -0.59728505,  0.57309218],
       [ 0.36803491,  0.80015358,  0.47360802],
       [-0.74144074, -0.05481624,  0.6687756 ]])

In [10]:
t = np.array(t).reshape((3,1))
t

array([[-0.14371057],
       [-3.6296544 ],
       [ 2.1912692 ]])

In [11]:
p = np.concatenate([r,t],1)
p

array([[ 0.5610846 , -0.59728505,  0.57309218, -0.14371057],
       [ 0.36803491,  0.80015358,  0.47360802, -3.6296544 ],
       [-0.74144074, -0.05481624,  0.6687756 ,  2.1912692 ]])

In [13]:
def rodrigues(r: np.ndarray) -> np.ndarray:
    '''
        if mat, convert to vec.
        if vec, convert to mat.
    '''
    if r.size == 3: return R.from_rotvec(r.squeeze()).as_matrix()
    elif r.size == 4: return R.from_quat(r.squeeze()).as_matrix()
    else: return R.from_matrix(r).as_rotvec().reshape((3, 1))
    
def get_rot(transform: np.ndarray) -> np.ndarray:
    '''
        args
        transform: camera pose vec or mat
        
        return
        rotational matrix
    '''
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return rodrigues(transform[:3])
    elif transform.shape == (3, 4) or transform.shape == (4, 4): #rotation matrix
        return transform[:3, :3]
    elif len(transform.shape) == 3: #rotation matrix (w/ batch)
        return transform[:, :3, :3]
    else:
        print(
            'Invalid shape of input transform: {}'.format(transform.shape))
        return None

def get_tr(transform: np.ndarray) -> np.ndarray:
    if transform.size == 6:
        transform = transform.reshape((6, 1))
        return transform[3:6].reshape((3, 1))
    elif transform.shape == (3, 4) or transform.shape == (4, 4):
        return transform[:3, 3].reshape((3, 1))
    elif len(transform.shape) == 3:
        return transform[:, :3, 3].T.reshape((3, -1))
    else:
        print(
            'Invalid shape of input transform: {}'.format(transform.shape))
        return None

def inverse_transform(transform: np.ndarray) -> np.ndarray:
    R, tr = get_rot(transform), get_tr(transform)
    R_inv = R.transpose()
    tr_inv = -R_inv.dot(tr)
    if transform.size == 6:
        r_inv = rodrigues(R_inv)
        return np.concatenate((r_inv, tr_inv), axis=0) # (6, 1) vector
    else:
        return np.concatenate((R_inv, tr_inv), axis=1) # (3, 4) matrix

In [16]:
inv_p = inverse_transform(p)
inv_p

array([[ 0.5610846 ,  0.36803491, -0.74144074,  3.04116958],
       [-0.59728505,  0.80015358, -0.05481624,  2.93856192],
       [ 0.57309218,  0.47360802,  0.6687756 ,  0.33592546]])

In [17]:
rodrigues(get_rot(inv_p))

array([[ 0.3174113 ],
       [-0.78960721],
       [-0.57984367]])

In [18]:
get_tr(inv_p)

array([[3.04116958],
       [2.93856192],
       [0.33592546]])